<a href="https://colab.research.google.com/github/chloe-nguyenminh/Movie_Genre_Detector_Project/blob/dev/data_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import io
import sys
import os.path
import urllib.request
from tqdm import tqdm
from os import listdir
from PIL import Image
import glob

pd.set_option('display.max_colwidth', None)
np.set_printoptions(threshold=sys.maxsize)

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Summer_2023/Models/Poster_Models/post-process1.csv",encoding='ISO-8859-1')
df.head()

,imdbId,Imdb Link,Title,IMDB Score,Genre,Poster
0,114709,http://www.imdb.com/title/tt114709,Toy Story (1995),8.3,Animation|Adventure|Comedy,"https://images-na.ssl-images-amazon.com/images/M/MV5BMDU2ZWJlMjktMTRhMy00ZTA5LWEzNDgtYmNmZTEwZTViZWJkXkEyXkFqcGdeQXVyNDQ2OTk4MzI@._V1_UX182_CR0,0,182,268_AL_.jpg"
1,113497,http://www.imdb.com/title/tt113497,Jumanji (1995),6.9,Action|Adventure|Family,"https://images-na.ssl-images-amazon.com/images/M/MV5BZTk2ZmUwYmEtNTcwZS00YmMyLWFkYjMtNTRmZDA3YWExMjc2XkEyXkFqcGdeQXVyMTQxNzMzNDI@._V1_UY268_CR10,0,182,268_AL_.jpg"
2,113228,http://www.imdb.com/title/tt113228,Grumpier Old Men (1995),6.6,Comedy|Romance,"https://images-na.ssl-images-amazon.com/images/M/MV5BMjQxM2YyNjMtZjUxYy00OGYyLTg0MmQtNGE2YzNjYmUyZTY1XkEyXkFqcGdeQXVyMTQxNzMzNDI@._V1_UX182_CR0,0,182,268_AL_.jpg"
3,114885,http://www.imdb.com/title/tt114885,Waiting to Exhale (1995),5.7,Comedy|Drama|Romance,"https://images-na.ssl-images-amazon.com/images/M/MV5BMTczMTMyMTgyM15BMl5BanBnXkFtZTcwOTc4OTQyMQ@@._V1_UY268_CR4,0,182,268_AL_.jpg"
4,113041,http://www.imdb.com/title/tt113041,Father of the Bride Part II (1995),5.9,Comedy|Family|Romance,"https://images-na.ssl-images-amazon.com/images/M/MV5BOTEyNzg5NjYtNDU4OS00MWYxLWJhMTItYWU4NTkyNDBmM2Y0XkEyXkFqcGdeQXVyMTQxNzMzNDI@._V1_UX182_CR0,0,182,268_AL_.jpg"


In [ ]:
df.shape

(37471, 6)

In [ ]:
# Remove null entries
df = df.dropna(how='any')

In [ ]:
not_found = []
for index, row in tqdm(df.iterrows()):

    url = row['Poster']
    imdb_id = row['imdbId']

    file_path = "/content/drive/MyDrive/Summer_2023/Models/Poster_Models/Posters/" + str(imdb_id) + ".jpg"

    try:
        response = urllib.request.urlopen(url)
        data = response.read()
        file = open(file_path, 'wb')
        file.write(bytearray(data))
        file.close()
    except Exception as e:
        print(e)
        not_found.append(imdb_id)

print("Number of posters not found = ", len(not_found))
print("Following IMDB_ID posters were not found:", not_found)



In [ ]:
df_2 = df[~df['imdbId'].isin(not_found)]
df_2.shape

(35310, 6)

In [ ]:
# Check for corrupted files and remove
bad_images = []
for file in glob.glob("/content/drive/MyDrive/Summer_2023/Models/Poster_Models/Posters/*.jpg"):

    try:
        img = Image.open(file) # open image file
        img.verify() # verify its an image
    except (IOError, SyntaxError) as e:
        print('Bad file:', file)

        bad_images.append(file)

print("Number of corrupt files:", len(bad_images))

In [ ]:
df_2.head()

,imdbId,Imdb Link,Title,IMDB Score,Genre,Poster
0,114709,http://www.imdb.com/title/tt114709,Toy Story (1995),8.3,Animation|Adventure|Comedy,"https://images-na.ssl-images-amazon.com/images/M/MV5BMDU2ZWJlMjktMTRhMy00ZTA5LWEzNDgtYmNmZTEwZTViZWJkXkEyXkFqcGdeQXVyNDQ2OTk4MzI@._V1_UX182_CR0,0,182,268_AL_.jpg"
1,113497,http://www.imdb.com/title/tt113497,Jumanji (1995),6.9,Action|Adventure|Family,"https://images-na.ssl-images-amazon.com/images/M/MV5BZTk2ZmUwYmEtNTcwZS00YmMyLWFkYjMtNTRmZDA3YWExMjc2XkEyXkFqcGdeQXVyMTQxNzMzNDI@._V1_UY268_CR10,0,182,268_AL_.jpg"
2,113228,http://www.imdb.com/title/tt113228,Grumpier Old Men (1995),6.6,Comedy|Romance,"https://images-na.ssl-images-amazon.com/images/M/MV5BMjQxM2YyNjMtZjUxYy00OGYyLTg0MmQtNGE2YzNjYmUyZTY1XkEyXkFqcGdeQXVyMTQxNzMzNDI@._V1_UX182_CR0,0,182,268_AL_.jpg"
3,114885,http://www.imdb.com/title/tt114885,Waiting to Exhale (1995),5.7,Comedy|Drama|Romance,"https://images-na.ssl-images-amazon.com/images/M/MV5BMTczMTMyMTgyM15BMl5BanBnXkFtZTcwOTc4OTQyMQ@@._V1_UY268_CR4,0,182,268_AL_.jpg"
4,113041,http://www.imdb.com/title/tt113041,Father of the Bride Part II (1995),5.9,Comedy|Family|Romance,"https://images-na.ssl-images-amazon.com/images/M/MV5BOTEyNzg5NjYtNDU4OS00MWYxLWJhMTItYWU4NTkyNDBmM2Y0XkEyXkFqcGdeQXVyMTQxNzMzNDI@._V1_UX182_CR0,0,182,268_AL_.jpg"


In [ ]:
#Checking if all the imdb_id listed here actually have its poster image

image_list = []
i = 0
try:
  for file in glob.glob("/content/drive/MyDrive/Summer_2023/Models/Poster_Models/Posters/*.jpg"):
      image_list.append(file)
except Exception as e:
  print(e)

print("Number of files found:", len(image_list))

Streaming output truncated to the last 5000 lines.
29768
29769
29770
29771
29772
29773
29774
29775
29776
29777
29778
29779
29780
29781
29782
29783
29784
29785
29786
29787
29788
29789
29790
29791
29792
29793
29794
29795
29796
29797
29798
29799
29800
29801
29802
29803
29804
29805
29806
29807
29808
29809
29810
29811
29812
29813
29814
29815
29816
29817
29818
29819
29820
29821
29822
29823
29824
29825
29826
29827
29828
29829
29830
29831
29832
29833
29834
29835
29836
29837
29838
29839
29840
29841
29842
29843
29844
29845
29846
29847
29848
29849
29850
29851
29852
29853
29854
29855
29856
29857
29858
29859
29860
29861
29862
29863
29864
29865
29866
29867
29868
29869
29870
29871
29872
29873
29874
29875
29876
29877
29878
29879
29880
29881
29882
29883
29884
29885
29886
29887
29888
29889
29890
29891
29892
29893
29894
29895
29896
29897
29898
29899
29900
29901
29902
29903
29904
29905
29906
29907
29908
29909
29910
29911
29912
29913
29914
29915
29916
29917
29918
29919
29920
29921
29922
29923
29924
29925
2

In [ ]:
bad_images = []
for file in glob.glob("Posters/*.jpg"):
    try:
        img = Image.open(file) # open image file
        img.verify() # verify its an image
    except (IOError, SyntaxError) as e:
        print('Bad file:', file)

        bad_images.append(file)

print("Number of corrupt files:", len(bad_images))

Number of corrupt files: 0


In [ ]:
df_2.to_csv('/content/drive/MyDrive/Summer_2023/Models/Poster_Models/post-process2.csv', index=None)

In [ ]:
df2 = pd.read_csv('/content/drive/MyDrive/Summer_2023/Models/Poster_Models/post-process2.csv', delimiter=',')

In [ ]:
df2.shape

(35310, 6)

In [ ]:
df2.head(5)

,imdbId,Imdb Link,Title,IMDB Score,Genre,Poster
0,114709,http://www.imdb.com/title/tt114709,Toy Story (1995),8.3,Animation|Adventure|Comedy,"https://images-na.ssl-images-amazon.com/images/M/MV5BMDU2ZWJlMjktMTRhMy00ZTA5LWEzNDgtYmNmZTEwZTViZWJkXkEyXkFqcGdeQXVyNDQ2OTk4MzI@._V1_UX182_CR0,0,182,268_AL_.jpg"
1,113497,http://www.imdb.com/title/tt113497,Jumanji (1995),6.9,Action|Adventure|Family,"https://images-na.ssl-images-amazon.com/images/M/MV5BZTk2ZmUwYmEtNTcwZS00YmMyLWFkYjMtNTRmZDA3YWExMjc2XkEyXkFqcGdeQXVyMTQxNzMzNDI@._V1_UY268_CR10,0,182,268_AL_.jpg"
2,113228,http://www.imdb.com/title/tt113228,Grumpier Old Men (1995),6.6,Comedy|Romance,"https://images-na.ssl-images-amazon.com/images/M/MV5BMjQxM2YyNjMtZjUxYy00OGYyLTg0MmQtNGE2YzNjYmUyZTY1XkEyXkFqcGdeQXVyMTQxNzMzNDI@._V1_UX182_CR0,0,182,268_AL_.jpg"
3,114885,http://www.imdb.com/title/tt114885,Waiting to Exhale (1995),5.7,Comedy|Drama|Romance,"https://images-na.ssl-images-amazon.com/images/M/MV5BMTczMTMyMTgyM15BMl5BanBnXkFtZTcwOTc4OTQyMQ@@._V1_UY268_CR4,0,182,268_AL_.jpg"
4,113041,http://www.imdb.com/title/tt113041,Father of the Bride Part II (1995),5.9,Comedy|Family|Romance,"https://images-na.ssl-images-amazon.com/images/M/MV5BOTEyNzg5NjYtNDU4OS00MWYxLWJhMTItYWU4NTkyNDBmM2Y0XkEyXkFqcGdeQXVyMTQxNzMzNDI@._V1_UX182_CR0,0,182,268_AL_.jpg"


In [ ]:
image_paths = []
imdb_id = []
genres = []
titles = []
for file in glob.glob("/content/drive/MyDrive/Summer_2023/Models/Poster_Models/Posters/*.jpg"):
    ind = file[file.rfind('/') + 1 : file.find('.')]
    title = df[df["imdbId"] == int(ind)]["Title"].values[0]
    genre = df[df["imdbId"] == int(ind)]["Genre"].values[0]

    image_paths.append(file)
    imdb_id.append(id)
    genres.append(genre)
    titles.append(title)


df_2 = pd.DataFrame({'imdbId': imd_id, 'Genre': genres, 'Title': titles, 'Image_Paths': image_paths})


In [ ]:
columnsTitles = ['Image_Paths','imdbId','Genre','Title']
df_2 = df_2.reindex(columns=columnsTitles)
df_2.shape

(34767, 4)

In [ ]:
df_2.head()

,Image_Paths,imdbId,Genre,Title
0,/content/drive/MyDrive/Summer_2023/Models/Poster_Models/Posters/114709.jpg,79142,Animation|Adventure|Comedy,Toy Story (1995)
1,/content/drive/MyDrive/Summer_2023/Models/Poster_Models/Posters/113497.jpg,79142,Action|Adventure|Family,Jumanji (1995)
2,/content/drive/MyDrive/Summer_2023/Models/Poster_Models/Posters/113228.jpg,79142,Comedy|Romance,Grumpier Old Men (1995)
3,/content/drive/MyDrive/Summer_2023/Models/Poster_Models/Posters/114885.jpg,79142,Comedy|Drama|Romance,Waiting to Exhale (1995)
4,/content/drive/MyDrive/Summer_2023/Models/Poster_Models/Posters/113041.jpg,79142,Comedy|Family|Romance,Father of the Bride Part II (1995)


In [ ]:
df_2.to_csv("/content/drive/MyDrive/Summer_2023/Models/Poster_Models/post-process3.csv", index = None)

In [ ]:
df3 = pd.read_csv('/content/drive/MyDrive/Summer_2023/Models/Poster_Models/post-process3.csv', delimiter=',')

In [ ]:
accurate_imdbId = []
for path in df3['Image_Paths']:
  try:
    id = path[path.rfind('/')  + 1: path.find('.')]
    if not all(char.isnumeric for char in id):
      raise Exception
    else:
      accurate_imdbId.append(id)
  except Exception:
    print(id)

print(len(accurate_imdbId))

34767


In [ ]:
df3.columns
df3 = df3.drop(columns=['imdbId'])
df3.head(5)

,Image_Paths,Genre,Title
0,/content/drive/MyDrive/Summer_2023/Models/Poster_Models/Posters/114709.jpg,Animation|Adventure|Comedy,Toy Story (1995)
1,/content/drive/MyDrive/Summer_2023/Models/Poster_Models/Posters/113497.jpg,Action|Adventure|Family,Jumanji (1995)
2,/content/drive/MyDrive/Summer_2023/Models/Poster_Models/Posters/113228.jpg,Comedy|Romance,Grumpier Old Men (1995)
3,/content/drive/MyDrive/Summer_2023/Models/Poster_Models/Posters/114885.jpg,Comedy|Drama|Romance,Waiting to Exhale (1995)
4,/content/drive/MyDrive/Summer_2023/Models/Poster_Models/Posters/113041.jpg,Comedy|Family|Romance,Father of the Bride Part II (1995)


In [ ]:
df3.insert(1, "imdbId", accurate_imdbId)



In [ ]:
df3.head(5)


,Image_Paths,imdbId,Genre,Title
0,/content/drive/MyDrive/Summer_2023/Models/Poster_Models/Posters/114709.jpg,114709,Animation|Adventure|Comedy,Toy Story (1995)
1,/content/drive/MyDrive/Summer_2023/Models/Poster_Models/Posters/113497.jpg,113497,Action|Adventure|Family,Jumanji (1995)
2,/content/drive/MyDrive/Summer_2023/Models/Poster_Models/Posters/113228.jpg,113228,Comedy|Romance,Grumpier Old Men (1995)
3,/content/drive/MyDrive/Summer_2023/Models/Poster_Models/Posters/114885.jpg,114885,Comedy|Drama|Romance,Waiting to Exhale (1995)
4,/content/drive/MyDrive/Summer_2023/Models/Poster_Models/Posters/113041.jpg,113041,Comedy|Family|Romance,Father of the Bride Part II (1995)


In [ ]:
df3.to_csv("/content/drive/MyDrive/Summer_2023/Models/Poster_Models/post-process3.csv", index = None)

In [ ]:
df3 = pd.read_csv("/content/drive/MyDrive/Summer_2023/Models/Poster_Models/post-process3.csv", delimiter = ',')
df3.head(5)

,Image_Paths,imdbId,Genre,Title
0,/content/drive/MyDrive/Summer_2023/Models/Poster_Models/Posters/114709.jpg,114709,Animation|Adventure|Comedy,Toy Story (1995)
1,/content/drive/MyDrive/Summer_2023/Models/Poster_Models/Posters/113497.jpg,113497,Action|Adventure|Family,Jumanji (1995)
2,/content/drive/MyDrive/Summer_2023/Models/Poster_Models/Posters/113228.jpg,113228,Comedy|Romance,Grumpier Old Men (1995)
3,/content/drive/MyDrive/Summer_2023/Models/Poster_Models/Posters/114885.jpg,114885,Comedy|Drama|Romance,Waiting to Exhale (1995)
4,/content/drive/MyDrive/Summer_2023/Models/Poster_Models/Posters/113041.jpg,113041,Comedy|Family|Romance,Father of the Bride Part II (1995)


In [ ]:
# Investigating which of the Imdb_Id of the original csv, did not have a corresponding image

Original_ID_list = np.asarray(df['imdbId']).astype(int)
ID_with_images_found = np.asarray(df3['imdbId']).astype(int)

#Original lengths of the two arrays
print("ORIGINAL:", len(Original_ID_list))
print("TRIMMED:", len(ID_with_images_found))

#First compare the unique values of these two arrays
uniq1, counts1 = np.unique(Original_ID_list, return_counts=True)
print("#unique ID values in ORIGINAL:", len(uniq1))

uniq2, counts2 = np.unique(ID_with_images_found, return_counts=True)
print("#unique ID values in TRIMMED:", len(uniq2))

ORIGINAL: 37471
TRIMMED: 34767
#unique ID values in ORIGINAL: 36879
#unique ID values in TRIMMED: 34767


In [ ]:
#Find the ID values which are non-unique
non_unique_id = []
for i in range(len(counts1)):

    if (counts1[i]>1):
        #print(uniq1[i], "with freq=", counts1[i])
        non_unique_id.append(uniq1[i])

#print(non_unique_id)

#Find rows where the column value of imdb is among these non_unique_id values
for i in range(len(non_unique_id)):

    print(df.loc[df['imdbId'] == int(non_unique_id[i])])

Streaming output truncated to the last 5000 lines.
10472  103805  http://www.imdb.com/title/tt103805  Beyond the Law (1993)   
11065  103805  http://www.imdb.com/title/tt103805  Beyond the Law (1993)   

       IMDB Score                 Genre  \
10472         6.6  Crime|Drama|Thriller   
11065         6.6  Crime|Drama|Thriller   

                                                                                                                                  Poster  
10472  https://images-na.ssl-images-amazon.com/images/M/MV5BMjE3MDQxNTkxNF5BMl5BanBnXkFtZTcwNzYzMzgxMQ@@._V1_UY268_CR3,0,182,268_AL_.jpg  
11065  https://images-na.ssl-images-amazon.com/images/M/MV5BMjE3MDQxNTkxNF5BMl5BanBnXkFtZTcwNzYzMzgxMQ@@._V1_UY268_CR3,0,182,268_AL_.jpg  
       imdbId                           Imdb Link                 Title  \
10619  106579  http://www.imdb.com/title/tt106579  Clean, Shaven (1993)   
11212  106579  http://www.imdb.com/title/tt106579  Clean, Shaven (1993)   

       IMDB Score      